unsupervised learning and clustered by average ratings and publishing years. This could be used to identify trends in categories and mechanics and make accurate predictions about games that will be released in the near future.

I use standard deviations a lot as well to help sort through all the game ratings. There are a lot of highly rated games that also have high standard deviations and speaking very generally...many of these games are a little harder to get on the table because they may be heavier,longer playing,complicated etc. 

In general the games that seem to be easier to get on the table with a wider group of individuals are those games with reasonably high ratings and also lower standard deviations. There are of course exceptions as many groups prefer games that may be heavier or are more tolerant of longer playing games for example.

All in all...I have had good luck making purchase decisions based on the higher rated games with lower deviations and I've avoided some other high rated games with higher SD's after I read comments and reviews and came to the conclusion that some of them would not be good choices for the players I game with.

This is the primary reason I like Tim Benjamin's FORM metric, which incorporates standard deviation and ratings (as well as number of ratings). While those more familiar with statistics can argue its merit as a statistical device, I find it gives me an excellent starting point when looking to find new games. Sure, there are exceptions, but so far, I've been happy with what I've chosen.

As for the initial inquiry, it does take some work, but you can get a standard deviation for every game in the database. Under Advanced Search, don't enter anything and press submit. You will get every game in the database with Avg, B. Avg, BGG Avg, Std Dev, # ratings. You can copy and paste this into a spreadsheet to do whatever manipulations you are interested in. 

You could of course, set a rating range (say 7 - 10) to get higher rated games and a number of votes (say 150) to get games a lot of people rated. You will get the same information that you then can put into a spreadsheet and sort by standard deviation.

Overall, looking through the list has just reinforced my belief that the BGG rank is more a "critics choice" list and that popularity or gameplay enjoyment is more accurately reflected in ranking by number of copies owned.
https://boardgamegeek.com/browse/boardgame?sort=numowned&sortdir=desc

Number of copies owned is a decent measure for mainstream appeal, whereas BGG rank is a decent measure for how good a game is for a particular niche.

check what features have correlation with high critics choice ranking, and which ones with how many people own a game(owned)

high playcount-to-unique-users ratio games are probably games worth looking into.
Number of players who've recorded at least one play of a game in the past year is also a good indicator of what hits the table, regardless of rank or copies owned.
https://boardgamegeek.com/plays/bygame/subtype/All/start/2017-01-01/end/2018-01-01

do party games have low complexity rating? low standard dev? what is the relationship between games of a certain type and complexity rating and standard deviation?

corelation of playing time and different domains? age vs diff domains?

oldest games and their correlation with different features?

oldest games and their correlation with different features

oldest games and their correlation with different features

In [1]:
import time
start = time.time()

In [2]:
# Import the necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## Getting the top ranked games

In [3]:
# %%time
# top_games = pd.DataFrame()
    
# for i in range(1,164):
#     # requests library is used to send a GET request to the url specified to extract the required information
#     # Extract the required information from the given url
#     response = requests.get("https://www.boardgamegeek.com/browse/boardgame/page/"+str(i)+"?sort=rank")
#     content = response.content

#     # Parse the extracted information using Beautiful Soup library
#     parser = BeautifulSoup(content, 'html.parser')

#     # Extract the content with <body></body> tags from the HTML page
#     body = parser.body
    
#     # Extract the div with id "main_content"
#     divs = body.find_all("div",id="main_content")
    
#     # Extract the content with tags 'td' and class 'collection_rank' 
#     # to get the ranks of the games
#     coll_rank_td = body.find_all("td", class_ = "collection_rank")
    
#     coll_rank = []
#     for coll_text in coll_rank_td:
#         # strip the tags and the tabs and whitespaces and append the ranks to a list
#         coll_rank.append(int(coll_text.text.strip()))
    
#     # Extract the content with tags 'td' and class 'collection_objectname' 
#     # to get the page links of the games as well as the names of the game
#     coll_objname_td = body.find_all("td", class_ = "collection_objectname")
    
#     # Extract ids from the links in order to use them to make api calls
#     coll_link = []
#     for coll_text in coll_objname_td:
#         temp = coll_text.a['href'].replace("/boardgame/","")
#         temp = temp.split("/")
#         coll_link.append(int(temp[0]))
    
#     # Extract the names of the games into a list
#     coll_name = []
#     for coll_text in coll_objname_td:
#         coll_name.append(coll_text.a.text.strip())
        
#     # Add the names, ids and ranks of the top games into a temp dataframe
#     temp = pd.DataFrame({"Name":coll_name, "Rank": coll_rank, "ID":coll_link})
    
#     # Concatenate all the top ranking games into one dataframe
#     top_games = pd.concat([top_games, temp])
# top_games

In [4]:
# top_games

In [5]:
# top_games.to_csv("top_games.csv", index=False)

In [6]:
top_games = pd.read_csv("top_games.csv")

In [7]:
top_games.columns

Index(['Name', 'Rank', 'ID'], dtype='object')

In [60]:
# top_games.reset_index(inplace=True, drop=True)
top_games.head()

,Name,Rank,ID
0,Gloomhaven,1,174430
1,Pandemic Legacy: Season 1,2,161936
2,Through the Ages: A New Story of Civilization,3,182028
3,Terraforming Mars,4,167791
4,Twilight Struggle,5,12333


## Getting stats for all the games

In [10]:
# Games Domain types
# Abstract Strategy Games (like Chess or Go)
# Customizable Games (CCGs, CMGs, LCGs, etc)
# Thematic Games (emphasis on narrative)
# Family Games (fun for kids and adults)
# Children's Games (best for younger kids)
# Party Games (few rules, lots of laughs)
# Strategy Games (more complex games)
# Wargames (conflict simulation, etc.)
# set it up so that if a game is in a category- set its rank as value, otherwise set 0

In [16]:
# %%time

# game_stats = pd.read_csv("game_stats.csv")
# Different game domains in the database
domain_types = [ 'Abstract Strategy Games', 'Customizable Games', 'Thematic Games', 'Family Games',
"Children's Games", "Party Games", "Strategy Games", "Wargames"]

game_stats = pd.DataFrame()

count = 0.0

for id in top_games['ID']:
    count+=1
    print(count)    
    
    temp = {}
    # get request to extract data for a game with a given id
    response_stats = requests.get("https://boardgamegeek.com/xmlapi/boardgame/"+str(id)+"?&stats=1")
    print(id)
    content_stats = response_stats.content
    parser_stats = BeautifulSoup(content_stats, 'html.parser')
    
    temp['ID'] = id
    temp['Year Published'] = parser_stats.yearpublished.text
    temp['Min Players'] = parser_stats.minplayers.text
    temp['Max Players'] = parser_stats.maxplayers.text
    temp['Playing Time'] = parser_stats.playingtime.text
    temp['Min Playing Time'] = parser_stats.minplaytime.text
    temp['Max Playing Time'] = parser_stats.maxplaytime.text
    # Number of people who own the game
    temp['Owned'] = parser_stats.owned.text
    temp['Users Rated'] = parser_stats.usersrated.text
    temp['Average Rating'] = parser_stats.average.text
    temp['Bayes Average Rating'] = parser_stats.bayesaverage.text
    temp['Age'] = int(parser_stats.age.text)
    # ease of understanding the rules of the game
    temp['Complexity Rating'] = float(parser_stats.averageweight.text)
    # num of comments on the game's review page
    temp['Comments'] = parser_stats.numcomments.text
    # num of people wishing to acquire the game
    temp['Wishing'] = parser_stats.wishing.text
    # ??
    temp['NumWeights'] = parser_stats.numweights.text
    # num of people wanting to acquire the game
    temp['Wanting'] = parser_stats.wanting.text
    # number of people willing to trade te game for another game
    temp['Trading'] = parser_stats.trading.text
    
    # games that have a videogame based on it
    if (parser_stats.videogamebg != None):
        temp['Video Games'] = parser_stats.videogamebg.text
    else:
        temp['Video Games'] = ' '
    
    
    temp['Median'] = parser_stats.median.text
    # standard deviation rating of the game
    temp['Std Deviation'] = parser_stats.stddev.text
    
    # primary publisher of the board game
    temp['Publisher'] = parser_stats.findAll("boardgamepublisher")[0].text
    
    # other (regional) publishers of the game
    publishers = parser_stats.findAll("boardgamepublisher")
    game_publishers = []
    for publisher in publishers[1:]:
        game_publishers.append(publisher.text)
    temp['Secondary Publishers'] = game_publishers
    
    polls = parser_stats.findAll("poll")
    
    # ideal number of players for the game as suggessted by the user community
    # num players classified into three categories: best, recommended and not recommended
    poll_numplayers = parser_stats.find("poll",{"name":"suggested_numplayers"})
    
    players_df = pd.DataFrame()
    num = []
    labels = []
    percent = []
    for i in range(1,len(poll_numplayers.findAll("results"))):
        poll_numplayers_1 = poll_numplayers.find("results",{"numplayers":i})
        numvotes_players_1 = []
        values_players_1 = []
        total = 0.0
        for result in poll_numplayers_1.findAll("result"):
            total += int(result['numvotes'])
            numvotes_players_1.append(int(result['numvotes']))
            values_players_1.append(result['value'])
        total = (1.0 if total == 0.0 else total)
        if numvotes_players_1 != []: 
            max_players_1 = max(numvotes_players_1)
            percent_players_1 = (max(numvotes_players_1)/total)*100
            index_players_1 = numvotes_players_1.index(max_players_1)
            num.append(i)
            labels.append(values_players_1[index_players_1])
            percent.append(percent_players_1)
    players_df['num'] = num
    players_df['labels'] = labels
    players_df['percent'] = percent
    
    for label in players_df['labels'].unique():
        temp['Num Players '+label] = (players_df[players_df['labels'] == label]['num']).tolist()
    
    # amount of info (about the rules) required to play the game
    poll_lang = parser_stats.find("poll",{"name":"language_dependence"})
    numvotes_lang = []
    values_lang = []
    for result in poll_lang.findAll("result"):
        numvotes_lang.append(int(result['numvotes']))
        values_lang.append(result['value'])
    if numvotes_lang != []:
        max_lang = max(numvotes_lang)
        index_lang = numvotes_lang.index(max_lang)
        temp['Language Dependence'] = values_lang[index_lang]
    else:
        temp['Language Dependence'] = ' '
    
    # user suggested age to play the game
    poll_age = parser_stats.find("poll",{"name":"suggested_playerage"})
    numvotes_age = []
    values_age = []
    for result in poll_age.findAll("result"):
        numvotes_age.append(int(result['numvotes']))
        values_age.append(result['value'])
    if numvotes_age !=[]:
        max_age = max(numvotes_age)
        index_age = numvotes_age.index(max_age)
        temp['Suggested Player Age'] = values_age[index_age]
    else:
        temp['Suggested Player Age'] = 0
    
    # domain types of the game
    domains = []
    boardgamesubdomain = parser_stats.findAll("boardgamesubdomain")
    for domain in boardgamesubdomain:
        domains.append(domain.text)
    temp['Game Domains'] = domains
    
    for domain in domain_types:
        if domain in domains:
            temp[domain] = True
        else:
            temp[domain] = False
    
    # categories the games fall in
    categories = []
    gamecategory = parser_stats.findAll("boardgamecategory")
    for category in gamecategory:
        categories.append(category.text)
    temp['Game Categories'] = categories
    
    # game mechanics
    mechanics = parser_stats.findAll("boardgamemechanic")
    game_mechanics = []
    for mechanic in mechanics:
        game_mechanics.append(mechanic.text)
    temp['Game Mechanics'] = game_mechanics

    # game designers
    designers = parser_stats.findAll("boardgamedesigner")
    game_designers = []
    for designer in designers:
        game_designers.append(designer.text)
    temp['Game Designers'] = game_designers

    # board game family
    families = parser_stats.findAll("boardgamefamily")
    game_families = []
    for family in families:
        game_families.append(family.text)
    temp['Game Family'] = game_families
    
    # ranks of the different game domains
    ranks = parser_stats.findAll("rank")
# Should I keep this column in the df or not?
#     rank_dict={}
#     for rank in ranks:
#         rank_dict[rank['friendlyname']]=int(rank['value'])
#     temp['Ranks'] = rank_dict

    if len(ranks)>1:
        for rank in ranks[1:]:
            temp[rank['friendlyname']]=int(rank['value'])
    
    # game artists
    artists = parser_stats.findAll("boardgameartist")
    game_artists = []
    for artist in artists:
        game_artists.append(artist.text)
    temp['Game Artists'] = game_artists
    
    # converting the dict to a dataframe
    temp_df = pd.DataFrame.from_dict(temp, orient='index')
    temp_df = temp_df.transpose()
    
    game_stats = pd.concat([game_stats, temp_df],sort=False)
# game_stats


1.0
204461
2.0
35311
3.0
11964
4.0
22758
5.0
143131
6.0
161126
7.0
145390
8.0
120886
9.0
205544
10.0
1986
11.0
25344
12.0
5494
13.0
24698
14.0
27480
15.0
148282
16.0
228425
17.0
183360
18.0
219766
19.0
17433
20.0
26372
21.0
1294
22.0
163481
23.0
177566
24.0
193122
25.0
230968
26.0
15471
27.0
171721
28.0
85340
29.0
68227
30.0
197377
31.0
71569
32.0
746
33.0
1907
34.0
75529
35.0
128154
36.0
22901
37.0
144460
38.0
137173
39.0
168265
40.0
129013
41.0
8354
42.0
18687
43.0
184917
44.0
14685
45.0
3965
46.0
224830
47.0
129309
48.0
93733
49.0
118330
50.0
83919
51.0
1940
52.0
234946
53.0
182801
54.0
1305
55.0
8359
56.0
217338
57.0
163186
58.0
145381
59.0
174990
60.0
89880
61.0
132229
62.0
5479
63.0
22203
64.0
21286
65.0
26855
66.0
2194
67.0
7033
68.0
131060
69.0
65061
70.0
21915
71.0
130556
72.0
189453
73.0
26999
74.0
246855
75.0
158891
76.0
22673
77.0
16105
78.0
20122
79.0
6374
80.0
169555
81.0
31605
82.0
60579
83.0
3100
84.0
33984
85.0
253379
86.0
81297
87.0
5990
88.0
7752
89.0
10995
90.0
1802

1218
686.0
4107
687.0
11756
688.0
124290
689.0
194387
690.0
13584
691.0
12900
692.0
3405
693.0
6118
694.0
213421
695.0
9509
696.0
23654
697.0
150220
698.0
11957
699.0
6017
700.0
11132
701.0
7870
702.0
183402
703.0
215455
704.0
97521
705.0
125028
706.0
111142
707.0
4294
708.0
82976
709.0
5107
710.0
127650
711.0
249410
712.0
17073
713.0
3054
714.0
9577
715.0
2230
716.0
12211
717.0
5618
718.0
196031
719.0
7640
720.0
4327
721.0
7934
722.0
190758
723.0
573
724.0
130764
725.0
2405
726.0
107703
727.0
31074
728.0
2546
729.0
246812
730.0
180674
731.0
172557
732.0
39382
733.0
164590
734.0
99791
735.0
134631
736.0
186265
737.0
73589
738.0
222951
739.0
20774
740.0
1598
741.0
9146
742.0
86919
743.0
18205
744.0
18265
745.0
84671
746.0
182228
747.0
6848
748.0
59061
749.0
98261
750.0
22511
751.0
136085
752.0
20043
753.0
4141
754.0
29375
755.0
93879
756.0
121076
757.0
186435
758.0
140468
759.0
229527
760.0
7447
761.0
223455
762.0
23550
763.0
205419
764.0
108679
765.0
20966
766.0
25462
767.0
37260
768.0

158236
1334.0
66517
1335.0
155700
1336.0
177868
1337.0
757
1338.0
96188
1339.0
219217
1340.0
240100
1341.0
15211
1342.0
7980
1343.0
166710
1344.0
129311
1345.0
204397
1346.0
183716
1347.0
114
1348.0
6699
1349.0
219622
1350.0
183340
1351.0
1043
1352.0
244144
1353.0
130725
1354.0
194640
1355.0
116231
1356.0
25014
1357.0
41757
1358.0
184460
1359.0
254135
1360.0
207378
1361.0
25866
1362.0
231878
1363.0
178285
1364.0
6720
1365.0
138201
1366.0
86006
1367.0
178154
1368.0
181574
1369.0
8283
1370.0
192344
1371.0
36478
1372.0
172837
1373.0
112445
1374.0
102150
1375.0
112557
1376.0
158604
1377.0
104775
1378.0
147465
1379.0
83907
1380.0
14514
1381.0
7830
1382.0
243752
1383.0
205039
1384.0
223763
1385.0
226605
1386.0
6978
1387.0
98475
1388.0
18880
1389.0
10267
1390.0
8544
1391.0
27682
1392.0
227527
1393.0
196392
1394.0
984
1395.0
1687
1396.0
2409
1397.0
237217
1398.0
2600
1399.0
120781
1400.0
10589
1401.0
175867
1402.0
4349
1403.0
18248
1404.0
9339
1405.0
205961
1406.0
11135
1407.0
248909
1408.0
14

68606
1957.0
13881
1958.0
138682
1959.0
35813
1960.0
69
1961.0
124918
1962.0
17649
1963.0
39279
1964.0
173408
1965.0
58707
1966.0
226
1967.0
171978
1968.0
10221
1969.0
52332
1970.0
201668
1971.0
222885
1972.0
11674
1973.0
209741
1974.0
2061
1975.0
37741
1976.0
37370
1977.0
176887
1978.0
29736
1979.0
156836
1980.0
871
1981.0
201006
1982.0
21269
1983.0
6907
1984.0
154681
1985.0
3485
1986.0
191364
1987.0
258104
1988.0
143703
1989.0
1029
1990.0
177477
1991.0
169550
1992.0
10557
1993.0
933
1994.0
211996
1995.0
2751
1996.0
220541
1997.0
174049
1998.0
7142
1999.0
11188
2000.0
3022
2001.0
5346
2002.0
11357
2003.0
57310
2004.0
148138
2005.0
200096
2006.0
220199
2007.0
38825
2008.0
13742
2009.0
168834
2010.0
8743
2011.0
1110
2012.0
165737
2013.0
144412
2014.0
19732
2015.0
151972
2016.0
232478
2017.0
256438
2018.0
42498
2019.0
150599
2020.0
11983
2021.0
193831
2022.0
167185
2023.0
79413
2024.0
31887
2025.0
35137
2026.0
217298
2027.0
18083
2028.0
465
2029.0
709
2030.0
125148
2031.0
86882
2032.0
23

131346
2580.0
124248
2581.0
130223
2582.0
112840
2583.0
147931
2584.0
3102
2585.0
165640
2586.0
533
2587.0
3979
2588.0
8979
2589.0
28522
2590.0
2205
2591.0
1268
2592.0
67034
2593.0
96672
2594.0
1875
2595.0
9913
2596.0
10394
2597.0
86433
2598.0
519
2599.0
195180
2600.0
8101
2601.0
36554
2602.0
1896
2603.0
93555
2604.0
5492
2605.0
185244
2606.0
25246
2607.0
6088
2608.0
197069
2609.0
23920
2610.0
165190
2611.0
193695
2612.0
195489
2613.0
21634
2614.0
191681
2615.0
10716
2616.0
109005
2617.0
35354
2618.0
79216
2619.0
163062
2620.0
171889
2621.0
906
2622.0
68340
2623.0
137156
2624.0
5303
2625.0
94476
2626.0
2362
2627.0
116735
2628.0
166977
2629.0
208323
2630.0
122867
2631.0
234101
2632.0
34692
2633.0
6748
2634.0
234439
2635.0
200396
2636.0
24994
2637.0
209383
2638.0
192352
2639.0
147616
2640.0
6042
2641.0
7622
2642.0
5321
2643.0
1869
2644.0
188753
2645.0
5881
2646.0
8363
2647.0
10264
2648.0
7921
2649.0
182276
2650.0
25569
2651.0
187000
2652.0
17836
2653.0
6478
2654.0
105004
2655.0
8201
2656

...

11703
5097.0
3894
5098.0
2178
5099.0
2127
5100.0
603
5101.0
575
5102.0
27712
5103.0
25642
5104.0
14343
5105.0
10543
5106.0
211168
5107.0
42892
5108.0
36234
5109.0
32109
5110.0
4221
5111.0
223034
5112.0
9319
5113.0
2116
5114.0
23292
5115.0
4614
5116.0
29080
5117.0
13464
5118.0
8670
5119.0
15214
5120.0
2972
5121.0
3693
5122.0
7711
5123.0
32245
5124.0
48154
5125.0
148179
5126.0
9471
5127.0
3374
5128.0
34014
5129.0
146496
5130.0
1138
5131.0
5012
5132.0
124393
5133.0
8178
5134.0
217671
5135.0
2760
5136.0
36560
5137.0
4219
5138.0
136585
5139.0
18163
5140.0
137354
5141.0
7541
5142.0
17738
5143.0
11347
5144.0
231362
5145.0
3156
5146.0
131530
5147.0
8253
5148.0
178028
5149.0
4022
5150.0
147396
5151.0
3144
5152.0
93594
5153.0
51067
5154.0
15828
5155.0
7421
5156.0
48867
5157.0
179124
5158.0
159507
5159.0
57276
5160.0
9049
5161.0
148442
5162.0
121790
5163.0
165551
5164.0
2680
5165.0
191077
5166.0
58139
5167.0
3091
5168.0
18262
5169.0
122711
5170.0
7553
5171.0
4426
5172.0
126464
5173.0
154585
5174.

32682
5735.0
33646
5736.0
30239
5737.0
3646
5738.0
6689
5739.0
119165
5740.0
6000
5741.0
2637
5742.0
137244
5743.0
28755
5744.0
55597
5745.0
28258
5746.0
187719
5747.0
132717
5748.0
8456
5749.0
15009
5750.0
23456
5751.0
25986
5752.0
12133
5753.0
3915
5754.0
148973
5755.0
22799
5756.0
2854
5757.0
10104
5758.0
2175
5759.0
1306
5760.0
2089
5761.0
5890
5762.0
5798
5763.0
6718
5764.0
116596
5765.0
2577
5766.0
13284
5767.0
5561
5768.0
184383
5769.0
10574
5770.0
28444
5771.0
3701
5772.0
4383
5773.0
19914
5774.0
16649
5775.0
2933
5776.0
4441
5777.0
15738
5778.0
11483
5779.0
128885
5780.0
12920
5781.0
163576
5782.0
191657
5783.0
9507
5784.0
194032
5785.0
7827
5786.0
15396
5787.0
19876
5788.0
1297
5789.0
32495
5790.0
246701
5791.0
7256
5792.0
192801
5793.0
21705
5794.0
41186
5795.0
119474
5796.0
138748
5797.0
9968
5798.0
2819
5799.0
141594
5800.0
1031
5801.0
17005
5802.0
5789
5803.0
7868
5804.0
237329
5805.0
140153
5806.0
172506
5807.0
17166
5808.0
88167
5809.0
16914
5810.0
5966
5811.0
3273
5812

57422
6374.0
28301
6375.0
27552
6376.0
4853
6377.0
39276
6378.0
10565
6379.0
89976
6380.0
3298
6381.0
36380
6382.0
21807
6383.0
3884
6384.0
156930
6385.0
4023
6386.0
6681
6387.0
762
6388.0
11563
6389.0
19425
6390.0
41890
6391.0
7611
6392.0
9578
6393.0
3080
6394.0
13115
6395.0
4042
6396.0
23598
6397.0
11528
6398.0
126443
6399.0
89956
6400.0
8448
6401.0
595
6402.0
2925
6403.0
31235
6404.0
172237
6405.0
18064
6406.0
24706
6407.0
146104
6408.0
4736
6409.0
6874
6410.0
4815
6411.0
8100
6412.0
5084
6413.0
186295
6414.0
2193
6415.0
124758
6416.0
101718
6417.0
5886
6418.0
32107
6419.0
1802
6420.0
177751
6421.0
6386
6422.0
17159
6423.0
175879
6424.0
5347
6425.0
408
6426.0
63900
6427.0
808
6428.0
8966
6429.0
194575
6430.0
62478
6431.0
10432
6432.0
12304
6433.0
10999
6434.0
8429
6435.0
3268
6436.0
6006
6437.0
3253
6438.0
298
6439.0
12491
6440.0
15806
6441.0
3622
6442.0
63443
6443.0
4325
6444.0
2430
6445.0
1672
6446.0
179876
6447.0
21622
6448.0
3753
6449.0
1973
6450.0
1221
6451.0
147423
6452.0
5121

13260
7020.0
39564
7021.0
9475
7022.0
622
7023.0
6994
7024.0
32993
7025.0
4070
7026.0
9160
7027.0
1488
7028.0
21529
7029.0
7860
7030.0
32601
7031.0
28674
7032.0
8295
7033.0
7249
7034.0
9559
7035.0
4076
7036.0
27800
7037.0
17687
7038.0
6245
7039.0
13165
7040.0
9725
7041.0
25167
7042.0
5996
7043.0
132203
7044.0
10808
7045.0
10744
7046.0
117910
7047.0
3849
7048.0
6684
7049.0
16530
7050.0
6486
7051.0
33799
7052.0
1096
7053.0
6228
7054.0
5051
7055.0
3830
7056.0
13714
7057.0
1934
7058.0
1249
7059.0
5512
7060.0
10518
7061.0
10414
7062.0
148273
7063.0
20028
7064.0
12601
7065.0
42686
7066.0
1703
7067.0
14361
7068.0
1443
7069.0
21749
7070.0
13208
7071.0
5663
7072.0
14275
7073.0
116821
7074.0
69676
7075.0
8813
7076.0
20790
7077.0
5675
7078.0
5166
7079.0
189489
7080.0
1653
7081.0
6397
7082.0
10625
7083.0
1255
7084.0
7457
7085.0
170954
7086.0
46746
7087.0
18618
7088.0
16620
7089.0
32157
7090.0
1625
7091.0
8454
7092.0
7547
7093.0
27675
7094.0
4064
7095.0
7605
7096.0
6398
7097.0
20624
7098.0
31068
70

2557
7667.0
5285
7668.0
136249
7669.0
1725
7670.0
1684
7671.0
8479
7672.0
15427
7673.0
26101
7674.0
19083
7675.0
629
7676.0
3676
7677.0
23631
7678.0
2308
7679.0
26137
7680.0
2841
7681.0
8009
7682.0
107971
7683.0
17211
7684.0
30640
7685.0
40421
7686.0
54559
7687.0
9173
7688.0
33198
7689.0
185
7690.0
2816
7691.0
5420
7692.0
14383
7693.0
53804
7694.0
9938
7695.0
1779
7696.0
3715
7697.0
22019
7698.0
21123
7699.0
43340
7700.0
142303
7701.0
5775
7702.0
22209
7703.0
8327
7704.0
164
7705.0
4781
7706.0
2206
7707.0
150690
7708.0
21734
7709.0
19001
7710.0
16461
7711.0
19274
7712.0
1333
7713.0
1424
7714.0
10043
7715.0
2717
7716.0
53376
7717.0
129368
7718.0
2957
7719.0
2322
7720.0
8104
7721.0
2120
7722.0
5018
7723.0
47082
7724.0
172032
7725.0
13373
7726.0
4282
7727.0
36611
7728.0
4852
7729.0
141636
7730.0
1531
7731.0
1629
7732.0
4476
7733.0
4597
7734.0
22980
7735.0
30255
7736.0
2707
7737.0
7399
7738.0
2518
7739.0
128185
7740.0
4470
7741.0
15599
7742.0
1174
7743.0
14708
7744.0
3320
7745.0
648
7746.0

In [17]:
%%time
game_stats.reset_index(drop=True, inplace=True)

CPU times: user 74 µs, sys: 0 ns, total: 74 µs
Wall time: 80.1 µs


In [18]:
game_stats.to_csv("game_stats1.csv", index=False)

In [ ]:
%%time

# merging the dataframe and removing duplicate columns
game_stats_merged = pd.merge(top_games, game_stats, how="inner")

In [55]:
# game_stats

In [21]:
end= time.time()
print(end-start)

9311.293476819992


In [52]:
game_stats_merged.to_csv("game_stats.csv", index=False)

In [ ]:
# game_stats[['Owned','Name', 'Num Players Best', 'Num Players Not Recommended', 'Num Players Recommended']]

In [54]:
game_stats_merged.columns

Index(['Name', 'Rank', 'ID', 'Year Published', 'Min Players', 'Max Players',
       'Playing Time', 'Min Playing Time', 'Max Playing Time', 'Owned',
       'Users Rated', 'Average Rating', 'Bayes Average Rating', 'Age',
       'Complexity Rating', 'Comments', 'Wishing', 'NumWeights', 'Wanting',
       'Trading', 'Video Games', 'Median', 'Std Deviation', 'Publisher',
       'Secondary Publishers', 'Num Players Recommended', 'Num Players Best',
       'Language Dependence', 'Suggested Player Age', 'Game Domains',
       'Abstract Strategy Games', 'Customizable Games', 'Thematic Games',
       'Family Games', 'Children's Games', 'Party Games', 'Strategy Games',
       'Wargames', 'Game Categories', 'Game Mechanics', 'Game Designers',
       'Game Family', 'Thematic Rank', 'Strategy Game Rank', 'Game Artists',
       'Num Players Not Recommended', 'War Game Rank', 'Customizable Rank',
       'Abstract Game Rank', 'Family Game Rank', 'Party Game Rank',
       'Children's Game Rank', 'RPG It

In [ ]:
# %%time
# response_stats = requests.get("https://boardgamegeek.com/xmlapi/boardgame/174430,161936,182028,167791,12333,187645,169786,220308,193738,120677?&stats=1 ")
# content_stats = response_stats.content
# content_stats

In [ ]:
# parser_stats = BeautifulSoup(content_stats, 'html.parser')

In [ ]:
# parser_stats

In [ ]:
# # ignore boardgames, boardgame, name, ratings, statistics, rank, boardgamecompilation, rpgpodcastepisode,
# # ??  description, thumbnail, image, boardgamepodcastepisode, boardgameaccessory, boardgameexpansion,
# # ?? boardgamehonor, boardgameimplementation, boardgameversion
# # yearpublished, minplayers, maxplayers, playingtime, minplaytime, maxplaytime, age, boardgamepublisher[0](primary), 
# # boardgamepublisher[1:](others), boardgamecategory, boardgameartist, boardgamemechanic, boardgamesubdomain, 
# # boardgamedesigner, boardgamefamily, ,poll_numplayers?, poll_lang_dep, poll_playerage, average, bayesaverage, 
# # ranks, owned, averageweight, stddev, wanting, wishing, numcomments, numweights, trading, usersrated, median, 
# # videogamebg

# tags = []
# for i in parser_stats():
#     tags.append(i.name)
# set(tags)

In [ ]:
# len(set(tags))

In [ ]:
parser_stats.findAll("rpgpodcastepisode")

In [ ]:
parser_stats.findAll("boardgamehonor")

In [ ]:
parser_stats.findAll("boardgameexpansion")

In [ ]:
parser_stats.findAll("boardgamecompilation")

In [ ]:
parser_stats.findAll("boardgameaccessory")

In [ ]:
parser_stats.findAll("boardgameimplementation")

In [ ]:
parser_stats.findAll("boardgameversion")

In [ ]:
end2= time.time()
print(end2-start)